In [ ]:
pip install lightfm

     |████████████████████████████████| 310 kB 5.9 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705355 sha256=0bc561bb2a3589ac4542f02d7958f64ce475db66e55f48a8e5c043721534c6ce
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [ ]:
pip install scikit-optimize

     |████████████████████████████████| 100 kB 4.9 MB/s 


In [ ]:
# import dependent libraries
import pandas as pd
import json
import os
import pickle
from scipy.sparse import csr_matrix
import numpy as np
import itertools
import math
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

In [ ]:
# folder = '/content/drive/MyDrive/datasets/Mind/'
folder = ''

In [ ]:
user_scored_answers = {'Hopeful':4, 'Happy':3, 'Coping':2, 
                      #  'Accepted': 4, 'Safe':5, 'Healthy':2
                       }
user_scored_answers

{'Coping': 2, 'Happy': 3, 'Hopeful': 4}

In [ ]:
user_scored_level = {k: ('high' if v >= 4 else 'medium' if v == 3 else 'low')
              for (k, v) in user_scored_answers.items()}

user_scored_level

{'Coping': 'low', 'Happy': 'medium', 'Hopeful': 'high'}

In [ ]:
with open(f'{folder}config.json') as f:
  config = json.load(f)

In [ ]:
levels = config['levels']

In [ ]:
models = pd.read_csv(f'{folder}models.csv')
# models.head()

,combination,Coping,Happy,Hopeful,number
0,"(1, 1, 1)",low,low,low,0
1,"(1, 1, 2)",low,low,medium,1
2,"(1, 1, 3)",low,low,high,2
3,"(1, 2, 1)",low,medium,low,3
4,"(1, 2, 2)",low,medium,medium,4


In [ ]:
models = models[(models['Coping']==user_scored_level['Coping'])
&(models['Happy']==user_scored_level['Happy'])
&(models['Hopeful']==user_scored_level['Hopeful'])]

,combination,Coping,Happy,Hopeful,number
5,"(1, 2, 3)",low,medium,high,5


In [ ]:
total_quest_comb = pd.read_csv(f'{folder}total_quest_comb.csv')
model_num = models['number'].tolist()[0]
quest_comb = total_quest_comb[total_quest_comb['model']==model_num]
quest_comb.head()

,combinations,questions,avg_rating,quest_comb_id,model
630,"(26, 19, 9)",What ideas have you got on how to improve your...,3,0,5
631,"(26, 19, 10)",What ideas have you got on how to improve your...,1,1,5
632,"(26, 19, 11)",What ideas have you got on how to improve your...,3,2,5
633,"(26, 19, 12)",What ideas have you got on how to improve your...,1,3,5
634,"(26, 20, 9)",What ideas have you got on how to improve your...,1,4,5


In [ ]:
interactions = {'user_id':[1, 2, 1, 2, 2], 
                'quest_comb_id':[1, 2, 3, 4, 5], 
                'rating':[3, 4, 0, 1, 2]}

interactions_selected = pd.DataFrame(interactions)
user_interaction = pd.pivot_table(interactions_selected, 
                                  index='user_id', 
                                  columns='quest_comb_id', values='rating')

user_interaction = user_interaction.fillna(0)
user_interaction_csr = csr_matrix(user_interaction.values)
user_id = list(user_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1


In [ ]:
item_dict ={}

for k, i in enumerate(quest_comb.index):
    item_dict[k] = quest_comb.loc[i,'combinations']

quest_comb_metadata_transformed = pd.get_dummies(quest_comb,
                                                 columns = ['avg_rating'])

quest_comb_metadata_csr = csr_matrix(quest_comb_metadata_transformed.drop(['quest_comb_id',
                                                                           'combinations',
                                                                           'questions',
                                                                           'model'
                                                                           ], axis=1).values)

In [ ]:
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(user_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

In [ ]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict, quest_csr, threshold = 0,
                               nrec_items = 5, show = True):
    
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    
    scores = pd.Series(model.predict(user_x,
                                     np.arange(n_items), 
                                     item_features= quest_csr))
    
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("User: " + str(user_id))
        # print("Known Likes:")
        # counter = 1
        # for i in known_items:
        #     print(str(counter) + '- ' + i)
        #     counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            # print(str(counter) + '- ' + i)
            
            counter+=1
            quest = quest_comb[quest_comb['combinations']== i]['questions'].tolist()[0]
            print(quest)

In [ ]:
sample_recommendation_user(model, user_interaction, 2, user_dict, item_dict, quest_comb_metadata_csr)

User: 2

 Recommended Items:
What ideas have you got on how to improve your wellbeing in coping?; What can you do to improve your score in this happy?; How would you like me to help you maintain in hopeful?
What ideas have you got on how to improve your wellbeing in coping?; What can you do to improve your score in this happy?; Who could best help you to maintain a high wellbeing score?
